# Need to figure out how to confirm that the format is correct...

In [1]:
def dup_entries(df):
    duplicates = df[df.duplicated(subset=['racer_id', 'discipline'], keep=False)]
    
    if not duplicates.empty:
        print("Found duplicates for (racer_id, discipline):")
        return duplicates
    else:
        print("No duplicates found.")
        return None

# Create the connection

In [2]:
import pandas as pd
import sqlite3
from helper_functions import prep_race_results, upload_results, drop_nulls

# Set the database path to a location with write permissions
db_path = '../race_league_results.db'

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


# Upload timetrial data (just race results)

In [3]:
import pandas as pd
import numpy as np
from helper_functions import clean_string, RACE_RESULT_COL_ORDER, upload_results

In [5]:
result_cols = ["bib", "discipline", "racer_id", "run1", "run2"]
df_load = pd.read_csv("2025/241229FnGRaceSimResultsBoth.csv", header=None, names=result_cols)

In [6]:
df_load["racer_id"] = df_load["racer_id"].apply(clean_string)
dup_entries(df_load)
df_load.replace({'DNF': 9998,"DSQ": 9998, "DNS": 9999, pd.NA: 9999}, inplace=True)

No duplicates found.


In [7]:
df_load.head(2)

bib discipline      racer_id  run1  run2
0    1        SKI     alexhenry  9999  9999
1    2        SKI  tonypetrella  9999  9999

In [8]:
race_date = '12/29/2024'
race_id = 241229
description = '2025 Race Sim (time trials)'

df_load['run1'] = pd.to_numeric(df_load['run1']) # errors='coerce')
df_load['run2'] = pd.to_numeric(df_load['run2']) # errors='coerce')
df_load['best_time'] = df_load[['run1', 'run2']].min(axis=1)
df_load["points"] = np.nan
df_load["team"] = np.nan
df_load["tier"] = np.nan
df_load["race_id"] = race_id

In [9]:
df_load[RACE_RESULT_COL_ORDER].head(2)

racer_id discipline  team  tier    run1    run2  best_time  points  \
0     alexhenry        SKI   NaN   NaN  9999.0  9999.0     9999.0     NaN   
1  tonypetrella        SKI   NaN   NaN  9999.0  9999.0     9999.0     NaN   

   race_id  bib  
0   241229    1  
1   241229    2

In [12]:
#upload_results(
#    df_load[RACE_RESULT_COL_ORDER],
#    race_id,
#    race_date,
#    description,
#    conn
#)

# Update Racers table

In [1]:
import pandas as pd
import numpy as np
import sqlite3

# Set the database path to a location with write permissions
db_path = '../race_league_results.db'

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [2]:
sql = """
with base AS (SELECT a.racer_id, max(a.name) as name
FROM Teams AS a
LEFT JOIN Racers AS b
ON a.racer_id = b.racer_id
WHERE b.racer_id IS NULL
group by 1)
SELECT racer_id,
    substr(name, 1, instr(name, ' ') - 1) AS first_name,
    substr(name, instr(name, ' ') + 1) AS last_name,
    NULL as gender,
    NULL as birth_year
FROM base
"""
df = pd.read_sql(sql, conn)
df.head()

racer_id first_name   last_name gender birth_year
0        adamszakacs       Adam     Szakacs   None       None
1       andrewmoscoe     Andrew      Moscoe   None       None
2  antonioliberatore    Antonio  Liberatore   None       None
3        audunbreder      Audun      Breder   None       None
4        audunbredur      Audun      Bredur   None       None

In [3]:
df.shape

(53, 5)

In [4]:
#df.to_sql('Racers', conn, if_exists='append', index=False)
#conn.commit()

# Update team data (new year start list)!

In [7]:
import os 

os.getcwd()

'/Users/will/Documents/github/cartar/alpine-fng.github.io/data'

In [12]:
teams_df = pd.read_csv('2025/2025StartList.csv')
teams_df = drop_nulls(teams_df, "name")
teams_df["racer_id"] = teams_df["name"].apply(clean_string)
teams_df.columns = ["bib", "discipline", "name", "tier", "team", "racer_id"]
teams_df["year"] = 2025

In [13]:
teams_df

bib discipline              name  tier    team        racer_id  year
0     1        SKI       David Moull    11   Green      davidmoull  2025
1     2        SKI       Shanna Reid    11  Purple      shannareid  2025
2     3        SKI      Devesh Varma    11  Orange     deveshvarma  2025
3     4        SKI        Dave Graff    11   Green       davegraff  2025
4     5        SKI          JD Etter    11  Yellow         jdetter  2025
..  ...        ...               ...   ...     ...             ...   ...
95  180       SNBD       Thomas Daly    12    Blue      thomasdaly  2025
96  132       SNBD     Sheri Ramshaw    13    Blue    sheriramshaw  2025
97  104       SNBD  Jacob Max Farber    12    Pink  jacobmaxfarber  2025
98  105       SNBD    Steve Crawford    13  Orange   stevecrawford  2025
99  182       SNBD    Graham Ramshaw    12   Green   grahamramshaw  2025

[100 rows x 7 columns]

In [15]:
#teams_df.to_sql('Teams', conn, if_exists='append', index=False)
#conn.commit()

# Remove folks who aren't racing anymore from the teams!

In [7]:
sql = """
ALTER TABLE Teams
ADD COLUMN is_active BOOLEAN DEFAULT TRUE;
"""

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

In [15]:
#Ryan Wood #22 can't participate anymore. 
sql = """
UPDATE Teams
SET is_active = FALSE
where racer_id='ryanwood'
and year = 2025;
"""

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

sql = """
SELECT * FROM Teams
where racer_id = 'ryanwood'
"""
df = pd.read_sql_query(sql, conn)
df

bib discipline       name  tier  team  racer_id  year  is_active
0   22        SKI  Ryan Wood     9  Blue  ryanwood  2025          0

In [12]:
#Adam Grossman #43 is out for the season with an injury.  
sql = """
UPDATE Teams
SET is_active = FALSE
where racer_id='adamgrossman'
and year = 2025;
"""

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

sql = """
SELECT * FROM Teams
where racer_id='adamgrossman'
"""
df = pd.read_sql_query(sql, conn)
df

bib discipline           name  tier           team      racer_id  year  \
0   50        SKI  Adam Grossman     7  Adam Grossman  adamgrossman  2024   
1   43        SKI  Adam Grossman     6         Purple  adamgrossman  2025   

   is_active  
0          1  
1          0

In [7]:
#Mctaggart is out....
sql = """
UPDATE Teams
SET is_active = FALSE
where racer_id = 'michaelmctaggart'
And year = 2025;
"""
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

sql = """
SELECT * FROM Teams
where racer_id = 'michaelmctaggart'
And year = 2025;"""
df = pd.read_sql_query(sql, conn)
df

bib discipline               name  tier   team          racer_id  year  \
0   87        SKI  Michael McTaggart     1  Black  michaelmctaggart  2025   

   is_active  
0          0

# Fix team names:

In [17]:
foo = pd.read_sql_query("SELECT * FROM Teams WHERE team = 'Green' AND year = 2025 ORDER BY tier;", conn)
foo

bib discipline                name  tier   team           racer_id  year
0    84        SKI         Mclean Wood     1  Green         mcleanwood  2025
1    74        SKI      William Cartar     2  Green      williamcartar  2025
2    67        SKI          Brad Brock     3  Green          bradbrock  2025
3    63        SKI         Mark Laurie     4  Green         marklaurie  2025
4    57        SKI      Todd Robertson     5  Green      toddrobertson  2025
5    46        SKI        Craig Pitman     6  Green        craigpitman  2025
6    38        SKI           David Eva     7  Green           davideva  2025
7    30        SKI     Sarah Robertson     8  Green     sarahrobertson  2025
8    19        SKI        Audun Breder     9  Green        audunbreder  2025
9    12        SKI  Antonio Liberatore    10  Green  antonioliberatore  2025
10    1        SKI         David Moull    11  Green         davidmoull  2025
11    4        SKI          Dave Graff    11  Green          davegraff  2025
12  182       SNBD      Graham Ramshaw    12  Green      grahamramshaw  2025

In [19]:
sql = """
UPDATE Teams
SET team = 'Orange Crush'
WHERE team = 'Orange'
AND year = 2025;
"""

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

In [18]:
sql = """
UPDATE Teams
SET team = 'HULK SMASH (Green)'
WHERE team = 'Green'
AND year = 2025;
"""

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

# First race upload 2025

In [3]:
path = '2025/250112FnGResultsBoth.csv'
race_date = '01/12/2025'
race_id = 250112
year = 2025
description = '2025 F&G Race #1'
#N_tiers=13
#N_teams=8 
max_points=10


In [4]:
results = prep_race_results(
    path,
    race_id,
    year,
    max_points,
    conn
)

/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tier_sorted.loc[tie_indices, 'points'] = tied_points
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:151: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[10.   5.   6.5  4.   3.   9.   6.5  8. ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['tier'] == tier, 'points'] = tier_sorted['points']
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.5' has dty

In [5]:
results.sort_values(by=["tier", "points"], ascending=[True, False], inplace=False).to_csv("test_results.csv")

In [6]:
# upload results:
#upload_results(results, race_id, race_date, description, conn)

# Upload second race

In [3]:
path = '2025/250126FnGResultsBoth.csv'
race_date = '01/26/2025'
race_id = 250126
year = 2025
description = '2025 F&G Race #2'
max_points=10


In [4]:
results = prep_race_results(
    path,
    race_id,
    year,
    max_points,
    conn
)

/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tier_sorted.loc[tie_indices, 'points'] = tied_points
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tier_sorted.loc[tie_indices, 'points'] = tied_points
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly 

In [5]:
results.sort_values(by=["tier", "points"], ascending=[True, False], inplace=False).to_csv("test_results.csv")

In [6]:
# upload results: -> wait to upload for time confirmation from Will!!
upload_results(results, race_id, race_date, description, conn)

# Upload 3rd race

In [3]:
path = '2025/250210FnGResultsBoth.csv'
race_date = '02/10/2025'
race_id = 250210
year = 2025
description = '2025 F&G Race #3'
max_points=10


In [4]:
results = prep_race_results(
    path,
    race_id,
    year,
    max_points,
    conn
)

/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tier_sorted.loc[tie_indices, 'points'] = tied_points
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tier_sorted.loc[tie_indices, 'points'] = tied_points
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly 

In [5]:
results.sort_values(by=["tier", "points"], ascending=[True, False], inplace=False).to_csv("test_results.csv")

In [6]:
# upload results: -> wait to upload for time confirmation from Will!!
upload_results(results, race_id, race_date, description, conn)

# Upload 4th race

In [23]:
order_by, race_id, year = "team, tier", 250112, 2025

sql = f"""
SELECT 
    RANK() OVER (
        ORDER BY COALESCE(race.best_time,99999) ASC
    ) AS rank,
    CASE
        WHEN team.bib is NOT NULL THEN team.bib
        ELSE race.bib
    END AS bib
    ,
    CASE
        WHEN team.name is NOT NULL THEN team.name
        WHEN names.name is NOT NULL THEN names.name
        ELSE race.racer_id
    END AS name,
    race.racer_id,
    race.discipline,
    team,
    tier,
    run1,
    run2,
    best_time,
    points
FROM (
    select racer_id, discipline, team, tier, run1, run2, best_time, points, bib
    from RaceResults
    where race_id = {race_id}
) AS race 
LEFT JOIN (
    select bib, discipline, racer_id, name
    from Teams
    where year = {year}
    AND is_active = TRUE
) AS team
ON race.racer_id = team.racer_id
AND race.discipline = team.discipline
LEFT JOIN (
    SELECT racer_id, CONCAT(first_name, ' ', last_name) AS name
    from Racers
) AS names
ON race.racer_id = names.racer_id
ORDER BY rank;
"""

df = pd.read_sql_query(sql, conn)

In [24]:
df

rank   bib             name         racer_id discipline  \
0       1  78.0         Jeff Cox          jeffcox        SKI   
1       2  84.0      Mclean Wood       mcleanwood        SKI   
2       3  77.0        Mike Parr         mikeparr        SKI   
3       4  74.0   William Cartar    williamcartar        SKI   
4       5  76.0  Emma Williamson   emmawilliamson        SKI   
..    ...   ...              ...              ...        ...   
100    98   NaN  avg_tier_points  avg_tier_points       None   
101    98   NaN  avg_tier_points  avg_tier_points       None   
102    98   NaN  avg_tier_points  avg_tier_points       None   
103    98   NaN  avg_tier_points  avg_tier_points       None   
104    98   NaN  avg_tier_points  avg_tier_points       None   

                   team  tier   run1     run2  best_time  points  
0                   Red   2.0  24.95    25.47      24.95    10.0  
1    HULK SMASH (Green)   1.0  25.36  9999.00      25.36    10.0  
2                Purple   2.0  25.46    25.54      25.46     9.0  
3    HULK SMASH (Green)   2.0  25.74    25.49      25.49     8.0  
4                  Pink   2.0  25.50    26.05      25.50     7.0  
..                  ...   ...    ...      ...        ...     ...  
100              Yellow  13.0    NaN      NaN        NaN     9.0  
101                Pink  13.0    NaN      NaN        NaN     9.0  
102               Black   1.0    NaN      NaN        NaN     7.0  
103               Black  13.0    NaN      NaN        NaN     9.0  
104                Blue   9.0    NaN      NaN        NaN     7.0  

[105 rows x 11 columns]

In [ ]:
#where race.best_time is not null

# Delete a race:

In [9]:
sql = """
SELECT *
FROM Races
where race_id = 250112
"""
df = pd.read_sql_query(sql, conn)
df.head()

race_id            race_date       description
0   250112  2025-01-12 00:00:00  2025 F&G Race #1
1   250112  2025-01-12 00:00:00  2025 F&G Race #1

In [10]:
sql = """
DELETE FROM Races
where race_id = 250112;
"""

#cursor = conn.cursor()
#cursor.execute(sql)
#conn.commit()

In [11]:
sql = """
SELECT *
FROM Races
where race_id = 250112
"""
df = pd.read_sql_query(sql, conn)
df.head()

Empty DataFrame
Columns: [race_id, race_date, description]
Index: []

In [12]:
sql = """
SELECT *
FROM RaceResults
where race_id = 250112
"""
df = pd.read_sql_query(sql, conn)
df.head()

racer_id discipline                team  tier   run1   run2  \
0         davidmoull        SKI  HULK SMASH (Green)  11.0  38.27  37.49   
1          davegraff        SKI  HULK SMASH (Green)  11.0  35.79  34.61   
2  antonioliberatore        SKI  HULK SMASH (Green)  10.0  37.13  36.23   
3        audunbreder        SKI  HULK SMASH (Green)   9.0  30.39  29.97   
4     sarahrobertson        SKI  HULK SMASH (Green)   8.0  34.37  34.56   

   best_time  points  race_id   bib  
0      37.49     4.0   250112   1.0  
1      34.61     9.0   250112   4.0  
2      36.23     4.0   250112  12.0  
3      29.97    10.0   250112  19.0  
4      34.37     3.0   250112  30.0

In [13]:
sql = """
DELETE FROM RaceResults
where race_id = 250112;
"""

#cursor = conn.cursor()
#cursor.execute(sql)
#conn.commit()

In [14]:
sql = """
SELECT *
FROM RaceResults
where race_id = 250112
"""
df = pd.read_sql_query(sql, conn)
df.head()

Empty DataFrame
Columns: [racer_id, discipline, team, tier, run1, run2, best_time, points, race_id, bib]
Index: []

# Upload race data (calc points)
[previous]

In [ ]:
path = 'data/2024/240218FnGResultsBoth.csv'
race_date = '02/18/2024'
race_id = 240218
description = '2024 F&G Race #4'
N_tiers=13
N_teams=8 

""" # Comment out for now
upload_new_race_results(
    path,
    race_date,
    race_id,
    description,
    N_tiers,
    N_teams,
    conn
)
"""